In [7]:
## column additions
## elevation change -- SAEID 
## latitude - longitude change -- ALP
## continent - continent code -- ERIC
## country to country code -- SAEID/ALP
## airport to airport code -- ERIC
## flighttime -- SAEID 
## average speed flighttime/distance -- ALP
## company name -- ALP

In [8]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [9]:
import airportsdata
airports = airportsdata.load()

df1 = pd.DataFrame(airports).T.reset_index()
removeindexcolumn = df1.columns.tolist()
removeindexcolumn.remove('index')
df2 = df1[removeindexcolumn]
df2['iata'].str.contains('BER')
df2['iata']=df2['iata'].replace('BER','SXF')
df2[df2['iata']=='SXF']
df2.to_csv('data/airportdata.csv')

C:\Users\erick\AppData\Local\Temp\ipykernel_22588\2418746488.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['iata']=df2['iata'].replace('BER','SXF')


In [10]:
df_flightdata = pd.read_csv('data/Train.csv')
df_airportdata = pd.read_csv('data/airportdata.csv')

df_flightdata['STA'] = pd.to_datetime(df_flightdata['STA'], format='%Y-%m-%d %H.%M.%S')
df_flightdata['STD'] = pd.to_datetime(df_flightdata['STD'], format='%Y-%m-%d %H:%M:%S')
df_flightdata['DATOP'] = pd.to_datetime(df_flightdata['DATOP'], format='%Y-%m-%d')
df_flightdata['target'].astype('float')
df_airportdata['elevation'] = df_airportdata['elevation'].astype('float')
df_airportdata['lat'] = df_airportdata['lat'].astype(float)
df_airportdata['lon'] = df_airportdata['lon'].astype(float)

In [11]:
df_flightdata.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,22.0


In [12]:
df_depart = df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']]
df_arrive = df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']]

depart_column_names = df_depart.columns.tolist() 
new_depart_names = []
for i in depart_column_names:
    new_depart_names.append(i + '_depart')
df_depart= df_depart.set_axis(new_depart_names, axis='columns')

arrive_column_names = df_arrive.columns.tolist() 
new_arrive_names = []
for i in arrive_column_names:
    new_arrive_names.append(i + '_arrive')
df_arrive = df_arrive.set_axis(new_arrive_names, axis='columns')


In [13]:
df_flightdata.rename(columns = {'ARRSTN':'iata_arrive'}, inplace = True)
df_flightdata.rename(columns = {'DEPSTN':'iata_depart'}, inplace = True)
df_flightdata1 = df_flightdata.merge(df_arrive, on='iata_arrive', how= 'left')
df = df_flightdata1.merge(df_depart, on='iata_depart', how= 'left')
df.rename(columns = {'iata_arrive':'ARRSTN'}, inplace = True)
df.rename(columns = {'iata_depart':'DEPSTN'}, inplace = True)


In [14]:
df.isna().sum()
df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,country_arrive,elevation_arrive,lat_arrive,lon_arrive,country_depart,elevation_depart,lat_depart,lon_depart
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,260.0,TN,22.0,36.851002,10.22720,MA,656.0,33.367500,-7.58997
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,20.0,TN,22.0,36.851002,10.22720,IT,768.0,45.630600,8.72811
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0.0,TR,325.0,41.275333,28.75200,TN,22.0,36.851002,10.22720
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0.0,FR,90.0,47.153198,-1.61073,TN,19.0,33.875000,10.77550
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,22.0,DZ,82.0,36.691002,3.21541,TN,22.0,36.851002,10.22720


In [15]:
# FIX COLUMN NAMES Function - general function build out with time
def fix_column_names(input_dataset):
    infun_data = input_dataset
    # pull the starting columns to show at the end the transformation 
    starting_columns = list(infun_data.columns)
    # carry out the changes; remove spaces and hyphens, lower case everything
    infun_data.columns = infun_data.columns.str.replace(' ','_')
    infun_data.columns = infun_data.columns.str.lower()
    infun_data.columns = infun_data.columns.str.replace('-', '_')
    # pull the ending columns to show at the end the transformation 
    ending_columns = list(infun_data.columns)
    # create a dictionary of the old names vs the new names 
    dict_of_names = {'old column name': starting_columns, 'new column name': ending_columns}
    dataframe_of_column_names = df = pd.DataFrame(dict_of_names) 
    print(dataframe_of_column_names)
    return infun_data

df = fix_column_names(df)

     old column name   new column name
0                 ID                id
1              DATOP             datop
2              FLTID             fltid
3             DEPSTN            depstn
4             ARRSTN            arrstn
5                STD               std
6                STA               sta
7             STATUS            status
8                 AC                ac
9             target            target
10    country_arrive    country_arrive
11  elevation_arrive  elevation_arrive
12        lat_arrive        lat_arrive
13        lon_arrive        lon_arrive
14    country_depart    country_depart
15  elevation_depart  elevation_depart
16        lat_depart        lat_depart
17        lon_depart        lon_depart


In [16]:
# you must make the dataset equal to the function
# i.e. df = del_duplicates(df)
def del_duplicates(input_dataset):
    infun_data = input_dataset
    # check for duplicates 
    print(infun_data.duplicated().value_counts())
    # remove duplicates
    infun_data = infun_data.drop_duplicates()
    # reset index inplace
    infun_data.reset_index(inplace=True, drop=True)
    return infun_data
    
df = del_duplicates(df)

False    107833
Name: count, dtype: int64


In [17]:
# which are statistical continous values?
# which are statistical quantitative values?


# check the different columns
# is a column a unique identifier? 
# can it be dropped to a unique identifier table, 
# i.e. unique identifier and then name, address, ticket number etc. (info table)
# postcode/zipcode an example of a none unique identifier, things that can't be grouped
df.dtypes

def table_nuniques(input_dataset):
    nunique_dictionary = {'column name': [], 'nuniques': [], 'uniques':[], 'dtype' : []}
    infun_data = input_dataset
    columnlist = list(infun_data.columns)
    for i in columnlist: 
        nunique_dictionary['column name'].append(i)
        nunique_dictionary['nuniques'].append(infun_data[i].nunique())        
        nunique_dictionary['uniques'].append(infun_data[i].unique())
        nunique_dictionary['dtype'].append(infun_data[i].dtype)
    # print the dictionary as a dataframe
    return pd.DataFrame(nunique_dictionary)

table_nuniques(df)

,column name,nuniques,uniques,dtype
0,id,107833,"[train_id_0, train_id_1, train_id_2, train_id_...",object
1,datop,1011,"[2016-01-03 00:00:00, 2016-01-13 00:00:00, 201...",datetime64[ns]
2,fltid,1861,"[TU 0712 , TU 0757 , TU 0214 , TU 0480 , TU 03...",object
3,depstn,132,"[CMN, MXP, TUN, DJE, TLS, IST, ORY, MIR, BRU, ...",object
4,arrstn,128,"[TUN, IST, NTE, ALG, BCN, ORY, FCO, NCE, MRS, ...",object
5,std,81697,"[2016-01-03 10:30:00, 2016-01-13 15:05:00, 201...",datetime64[ns]
6,sta,85136,"[2016-01-03 12:55:00, 2016-01-13 16:55:00, 201...",datetime64[ns]
7,status,5,"[ATA, DEP, RTR, SCH, DEL]",object
8,ac,68,"[TU 32AIMN, TU 31BIMO, TU 736IOK, TU 320IMU, T...",object
9,target,968,"[260.0, 20.0, 0.0, 22.0, 53.0, 10.0, 15.0, 16....",float64


In [20]:
#  no na was found!!
# print(df.isnull().sum())
print((df == 0).sum())

id                      0
datop                   0
fltid                   0
depstn                  0
arrstn                  0
std                     0
sta                     0
status                  0
ac                      0
target              38168
country_arrive          0
elevation_arrive        0
lat_arrive              0
lon_arrive              0
country_depart          0
elevation_depart        0
lat_depart              0
lon_depart              0
dtype: int64
